In [325]:
Identifier_and_Name = {
    "Au-PeEL": "Ebook Library",
    "CaONFJC": "MyiLibrary",
    "CaPaEBR": "Ebrary",
    "DE-He213": "Springer-Verlag",
    "MiAaPQ": "ProQuest",
    "NhCcYBP": "YBP",
    "OCoLC": "OCLC",
    "SFPDA_MiAaPQ": "ProQuest",
    "YBPUID": "YBP ID",
    "MIL": "MyiLibrary",
    "EBR": "Ebrary",
    "EBL": "Ebook Library",
    "EBC": "Ebook Central",
    "CaBNVSL": "SIAM",
    "CaBNvSL": "SIAM",
    "DcWaAPA": "APA",
    "DNLM": "US Med Library",
    "FR-PaOEC": "OECD",
    "IN-ChSCO": "Scope e-Knowedge Center",
    "MiFhGG": "Gale",
    "StDuBDS": "Bibliographic Data Services",
    "UtOrBLW": "Backstage Library Works",
    "VaAlASP": "Alexander Street Press",
    "EBS": "EBSCO",
    "FANhCcYBP": "YBP (FA)",
    "UK-OxUP": "Oxford UP",
    "no label": "Unlabeled",
}

> Start by loading Alpeh Sequential text file into OpenRefine with column widths 10,6,2

Put the unique name of the OpenRefine project for the BIB records in the variable `BIB_Project_Name` and run the cell

In [341]:
BIB_Project_Name = "wiley_bib_3 8 2021 txt"

# Organize BIB Record

## Organize MARC Fields and Subfields
Copy Organize_Alpeh_Sequential_for_BIB.json and paste into OpenRefine

## Pivot Subfields into Columns
Use a filter on "No Field Number" to select the following subfields, invert, then remove all matching rows:

- 020$a
- 020$q
- 020$z
- 024$2
- 024$a
- 035$a
- 245$a
- 245$b
- 245$c
- 245$n
- 250$a
- 264$b
- 264$c
- 710$a
- 710$e
- 776$i
- 776$z
- 856$3
- 856$u
- 856$z
- 897$a
- 897$e

Open a text filter for "Fields," click on the number of choices to get the facet choices as TSV, paste into variable "Columns" below, and run cell

In [342]:
Columns = """
020$a	274
020$z	2
024$2	390
024$a	539
035$a	3319
245$a	3427
245$b	892
245$c	1393
245$n	63
250$a	278
264$b	3039
264$c	2124
776$i	1314
776$z	1538
856$3	1031
856$u	1770
856$z	392
Field #1 020$a	1118
Field #1 020$q	467
Field #1 020$z	33
Field #1 024$2	26
Field #1 024$a	36
Field #1 035$a	108
Field #1 264$b	454
Field #1 264$c	299
Field #1 776$i	889
Field #1 776$z	83
Field #1 856$3	2
Field #1 856$u	1657
Field #1 856$z	1653
Field #10 020$a	73
Field #10 020$q	70
Field #10 020$z	82
Field #10 856$u	76
Field #10 856$z	76
Field #11 020$a	41
Field #11 020$q	29
Field #11 020$z	34
Field #11 856$u	55
Field #11 856$z	55
Field #12 020$a	31
Field #12 020$q	23
Field #12 020$z	27
Field #12 856$u	43
Field #12 856$z	43
Field #13 020$a	13
Field #13 020$q	17
Field #13 020$z	21
Field #13 856$u	33
Field #13 856$z	33
Field #14 020$a	6
Field #14 020$q	7
Field #14 020$z	11
Field #14 856$u	27
Field #14 856$z	27
Field #15 020$a	1
Field #15 020$q	3
Field #15 020$z	7
Field #15 856$u	21
Field #15 856$z	21
Field #16 020$a	1
Field #16 020$q	2
Field #16 020$z	4
Field #16 856$u	14
Field #16 856$z	14
Field #17 020$z	2
Field #17 856$u	10
Field #17 856$z	10
Field #18 020$z	2
Field #18 856$u	2
Field #18 856$z	2
Field #19 020$z	1
Field #19 856$u	2
Field #19 856$z	2
Field #2 020$a	1126
Field #2 020$q	466
Field #2 020$z	24
Field #2 024$2	9
Field #2 024$a	36
Field #2 035$a	108
Field #2 264$b	352
Field #2 264$c	70
Field #2 776$i	298
Field #2 776$z	88
Field #2 856$3	3
Field #2 856$u	1655
Field #2 856$z	1651
Field #20 020$z	1
Field #20 856$u	2
Field #20 856$z	2
Field #21 020$z	1
Field #21 856$u	1
Field #21 856$z	1
Field #3 020$a	837
Field #3 020$q	454
Field #3 020$z	135
Field #3 035$a	93
Field #3 264$b	77
Field #3 264$c	1
Field #3 776$i	64
Field #3 776$z	9
Field #3 856$u	1030
Field #3 856$z	1030
Field #4 020$a	824
Field #4 020$q	448
Field #4 020$z	114
Field #4 035$a	2
Field #4 264$b	15
Field #4 776$i	13
Field #4 776$z	2
Field #4 856$u	634
Field #4 856$z	634
Field #5 020$a	440
Field #5 020$q	321
Field #5 020$z	347
Field #5 264$b	4
Field #5 776$i	3
Field #5 776$z	1
Field #5 856$u	436
Field #5 856$z	436
Field #6 020$a	463
Field #6 020$q	302
Field #6 020$z	289
Field #6 264$b	2
Field #6 856$u	303
Field #6 856$z	303
Field #7 020$a	212
Field #7 020$q	207
Field #7 020$z	282
Field #7 856$u	203
Field #7 856$z	203
Field #8 020$a	202
Field #8 020$q	162
Field #8 020$z	193
Field #8 856$u	148
Field #8 856$z	148
Field #9 020$a	86
Field #9 020$q	102
Field #9 020$z	151
Field #9 856$u	111
Field #9 856$z	111
"""

**Indicator columns deleted at this point--unsure how to handle/how or if to preserve beyond this point**

Copy Pivot_Fields_and_Values.json and paste into OpenRefine

In [343]:
Columns = Columns.split("\n")[1:-1] # The splice removes elements created by having the opening and closing quotes on their own lines
Column_Names = []
for Column in Columns:
    Column = Column.rstrip("1234567890")
    Column = Column.rstrip("\t")
    Column_Names.append(Column)

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    For_OpenRefine.write("[")
    for Column in Column_Names:
        Template_String = f"""
            {{
                "op": "core/text-transform",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Column}",
                "expression": "grel:row.record.cells[\\"{Column}\\"].value[0]",
                "onError": "keep-original",
                "repeat": false,
                "repeatCount": 10,
                "description": "Fill up and down values in ``{Column}``"
            }},
            {{
                "op": "core/blank-down",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Column}",
                "description": "Blank down cells in column ``{Column}``"
            }},
        """
        For_OpenRefine.write(Template_String)
    For_OpenRefine.write("]")

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

Copy Remove_Blank_Rows_After_Pivot.json and paste into OpenRefine

In [344]:

def Column_Order_as_Letter(MARC_Tag):
    if MARC_Tag == "020$a":
        return "A"
    elif MARC_Tag == "020$z":
        return "B"
    elif MARC_Tag == "776$z":
        return "C"
    elif MARC_Tag == "020$q":
        return "D"
    elif MARC_Tag == "776$i":
        return "E"
    elif MARC_Tag == "024$a":
        return "F"
    elif MARC_Tag == "024$2":
        return "G"
    elif MARC_Tag == "035$a":
        return "H"
    elif MARC_Tag == "710$a":
        return "I"
    elif MARC_Tag == "710$e":
        return "J"
    elif MARC_Tag == "897$a":
        return "K"
    elif MARC_Tag == "897$e":
        return "L"
    elif MARC_Tag == "856$u":
        return "M"
    elif MARC_Tag == "856$z":
        return "N"
    elif MARC_Tag == "856$3":
        return "O"
    elif MARC_Tag == "264$c":
        return "P"
    elif MARC_Tag == "264$b":
        return "Q"
    elif MARC_Tag == "245$c":
        return "R"
    elif MARC_Tag == "250$a":
        return "S"
    elif MARC_Tag == "245$n":
        return "T"
    elif MARC_Tag == "245$b":
        return "U"
    elif MARC_Tag == "245$a":
        return "V"
    # No "else" statement because if there's a problem here, it will require manual intervention

Column_Order = {
    "A": {},
    "B": {},
    "C": {},
    "D": {},
    "E": {},
    "F": {},
    "G": {},
    "H": {},
    "I": {},
    "J": {},
    "K": {},
    "L": {},
    "M": {},
    "N": {},
    "O": {},
    "P": {},
    "Q": {},
    "R": {},
    "S": {},
    "T": {},
    "U": {},
    "V": {},
}

# Create nested dictionary for ordering columns
for Column in Column_Names:
    if " " in Column: # Has field number
        Field_Number = int(Column.split(" ")[1][1:])
        Ordering_Letter = Column_Order_as_Letter(Column.split(" ")[2])
        Column_Order[Ordering_Letter][Field_Number] = Column
    else:
        Ordering_Letter = Column_Order_as_Letter(Column)
        Column_Order[Ordering_Letter][0] = Column

# Create markers to ensure that all first column in pivots exist
if not Column_Order["Q"].get(0): # If there's no column that's just "264$b"
    Column_Order["Q"][0] = "264$b_marker"
if not Column_Order["P"].get(0): # If there's no column that's jsut "264$c"
    Column_Order["p"][0] = "264$c_marker"
if not Column_Order["O"].get(0): # If there's no column that's just "856$u"
    Column_Order["O"][0] = "856$u_marker"
if not Column_Order["H"].get(0): # If there's no column that's just "035$a"
    Column_Order["H"][0] = "035$a_marker"
if not Column_Order["A"].get(0): # If there's no column that's just "020$a"
    Column_Order["A"][0] = "020$a_marker"

# Sort by field order within tags
for key, value in Column_Order.items():
    value = dict(sorted(value.items()))
    Column_Order[key] = value

# Create list of column values and markers for columns needed for pivots in order
Column_Order_Values = []
for Outer_Value in Column_Order.values():
    for Inner_Value in Outer_Value.values():
        Column_Order_Values.append(Inner_Value)

# Create dictionary for adding columns needed for pivots and list of existing columns for adding to JSON 
Pivot_Addition_Column_Indexes = {Column_Value[0:5]: Column_Order_Values.index(Column_Value)+2 for Column_Value in Column_Order_Values if "_marker" in Column_Value}
Column_Order_Values = ['"' + Column_Value + '"' for Column_Value in Column_Order_Values if "_marker" not in Column_Value] #if "_marker" not in Column_Value
Reordering_String = ",\n".join(Column_Order_Values)

# Create final JSON for ordering columns
with open('JSON_Output.txt', 'w') as For_OpenRefine:
    Template_String = f"""
        [
            {{
                "op": "core/column-addition",
                "description": "Create column ``Count``",
                "engineConfig": {{
                    "facets": [],
                    "mode": "record-based"
                }},
                "newColumnName": "Count",
                "columnInsertIndex": 1,
                "baseColumnName": "SYS Number",
                "expression": "grel:row.record.cells[\\"SYS Number\\"].value[0]+\\"-\\"+toString(row.index-row.record.fromRowIndex+1)",
                "onError": "set-to-blank"
            }},
            {{
                "op": "core/column-reorder",
                "columnNames": [
                    "SYS Number",
                    "Count",
                    {Reordering_String}
                ],
                "description": "Reorder columns"
            }}
    """
    For_OpenRefine.write(Template_String)
    for Column_Name, Column_Position in Pivot_Addition_Column_Indexes.items():
        Template_String = f"""
            {{
                "op": "core/column-addition",
                "engineConfig": {{
                    "facets": [],
                    "mode": "record-based"
                }},
                "baseColumnName": "SYS Number",
                "expression": "grel:null",
                "onError": "set-to-blank",
                "newColumnName": "{Column_Name}",
                "columnInsertIndex": {Column_Position},
                "description": "Create column ``{Column_Name}`` for pivoting"
            }}
        """
        For_OpenRefine.write(Template_String)
    For_OpenRefine.write("]")

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

Copy Organize_Rarely_Duplicated_Fields.json and paste into OpenRefine

## Organize Publisher and Pub Date Fields

**Multiple publisher fields handled by combining htem with `]|[` seperator; is there a better option?**

Copy Organize_Publisher_Fields.json and paste into OpenRefine

**PUB DATE ABLE TO HANDLE SAME DATE MULTIPLE TIMES--NEED TO FIGURE OUT STRATEGY FOR IF WE WANT TO PRESERVE BOTH DATES**

Copy Organize_Pub_Date_Fields.json and paste into OpenRefine

## Organize URLs

Copy Organize_URL_Fields.json and paste into OpenRefine
* If the row's value in "Fields" is null, the value in "Values" won't be retained through the pivot

Open a text filter for "Fields", perform any clustering if appropriate, click on the number of choices to get the facet choices as TSV, paste into variable "URL_Headers" below, and run cell


In [330]:
URL_Headers = """
University Press Scholarship Online	20
"""

Copy Pivot_Fields_and_Values.json and paste into OpenRefine

Set "i" to the column number of column "020$a" and run cell

In [331]:
URL_Headers = URL_Headers.split("\n")[1:-1] # The splice removes elements created by having the opening and closing quotes on their own lines
Header_Names = []
for Header in URL_Headers:
    Header = Header.rstrip("1234567890")
    Header = Header.rstrip("\t")
    Header_Names.append(Header)

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    For_OpenRefine.write("[")
    i = 8
    for Header in Header_Names:
        Template_String = f"""
            {{
                "op": "core/text-transform",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Header}",
                "expression": "grel:row.record.cells[\\"{Header}\\"].value[0]",
                "onError": "keep-original",
                "repeat": false,
                "repeatCount": 10,
                "description": "Fill up and down values in ``{Header}``"
            }},
            {{
                "op": "core/blank-down",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Header}",
                "description": "Blank down cells in column ``{Header}``"
            }},
            {{
                "op": "core/column-move",
                "columnName": "{Header}",
                "index": {i},
                "description": "Move column ``{Header}`` to position {i}"
            }}
        """
        For_OpenRefine.write(Template_String)
        i += 1
    For_OpenRefine.write("]")

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

Copy Remove_Blank_Rows_After_Pivot.json and paste into OpenRefine

**897 and 710 fields not handled**

## Organize Identifier Fields

Copy Organize_Identifier_Fields.json and paste into OpenRefine

Open a text filter for "Fields", click on the number of choices to get the facet choices as TSV, paste into variable "Identifier_List" below, and run cell

In [332]:
Identifier_List = """
NhCcYBP	15
no label	5
YBPUID	20
"""

Copy Pivot_Fields_and_Values.json and paste into OpenRefine

In [333]:
Identifier_List = Identifier_List.split("\n")[1:-1] # The splice removes elements created by having the opening and closing quotes on their own lines
Identifiers = []
for Identifier in Identifier_List:
    Identifier = Identifier.rstrip("1234567890")
    Identifier = Identifier.rstrip("\t")
    Identifiers.append(Identifier)

Old_and_New_Headers = dict()
for Identifier in Identifiers:
    if Identifier_and_Name.get(Identifier) is None:
        print(f"{Identifier} doesn't have a value. Add one to the dictionary Identifier_and_Name at the top of the notebook and rerun all cells.")
        continue
    Old_and_New_Headers[Identifier] = Identifier_and_Name.get(Identifier)

Values_as_List = [l_org_code for l_org_code in Old_and_New_Headers.values()]
Values_as_Set = set(Values_as_List)
if len(Values_as_Set) < len(Values_as_List):
    Duplicates_Check = dict()
    for Set_Value in Values_as_Set:
        Duplicates_Check[Set_Value] = 0
        for List_Value in Values_as_List:
            if Set_Value == List_Value:
                Duplicates_Check[Set_Value] +=1

    Repeated_Headers = []
    for Key, Value in Duplicates_Check.items():
        if Value > 1:
            Repeated_Headers.append(Key)
    print("The headers below were matched to multiple org codes in Identifier_List:")
    print(Repeated_Headers)


Set "i" to the column number of column "Author"

In [334]:

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    For_OpenRefine.write("[")
    i = 5
    for Key, Value in Old_and_New_Headers.items():
        Template_String = f"""
            {{
                "op": "core/column-addition",
                "engineConfig": {{
                "facets": [],
                "mode": "row-based"
                }},
                "baseColumnName": "{Key}",
                "expression": "grel:row.record.cells[\\"{Key}\\"].value[0]",
                "onError": "set-to-blank",
                "newColumnName": "{Value}",
                "columnInsertIndex": {i},
                "description": "Create column ``{Value}`` by filling up and down ``{Key}``"
            }},
            {{
                "op": "core/blank-down",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{Value}",
                "description": "Blank down cells in column ``{Value}``"
            }},
            {{
                "op": "core/column-removal",
                "columnName": "{Key}",
                "description": "Remove column ``{Key}``"
            }}
        """
        For_OpenRefine.write(Template_String)
        i += 1
    For_OpenRefine.write("]")

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

Copy Remove_Blank_Rows_After_Pivot.json and paste into OpenRefine

## Organise ISBNs

Copy Organize_ISBN_Fields_1.json and paste into OpenRefine

Perform clustering on column "ISBN Type"

Copy Organize_ISBN_Fields_2.json and paste into OpenRefine

Open a text filter for "ISBN Type", click on the number of choices to get the facet choices as TSV, paste into variable "ISBN_Fields" below, and run cell

In [335]:
ISBN_Types = """
eISBN 10	20
eISBN 10 no 2	3
eISBN 10 no 3	3
eISBN 13	20
eISBN 13 no 2	4
eISBN 13 no 3	3
EPUB eISBN 10	1
EPUB eISBN 13	1
HB ISBN 10	1
HB ISBN 13	3
ISBN 10	16
ISBN 10 no 2	2
ISBN 13	20
ISBN 13 no 2	4
PB ISBN 10	2
PB ISBN 13	2
"""

Copy Pivot_Fields_and_Values.json and paste into OpenRefine

In [336]:
ISBN_Types = ISBN_Types.split("\n")[1:-1] # The splice removes elements created by having the opening and closing quotes on their own lines
ISBN_Fields = []
for ISBN_Type in ISBN_Types:
    ISBN_Type = ISBN_Type.rstrip("1234567890")
    ISBN_Type = ISBN_Type.rstrip("\t")
    ISBN_Fields.append(ISBN_Type)

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    For_OpenRefine.write("[")
    for ISBN_Type in ISBN_Fields:
        Template_String = f"""
            {{
                "op": "core/text-transform",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{ISBN_Type}",
                "expression": "grel:row.record.cells[\\"{ISBN_Type}\\"].value[0]",
                "onError": "keep-original",
                "repeat": false,
                "repeatCount": 10,
                "description": "Fill up and down values in ``{ISBN_Type}``"
            }},
            {{
                "op": "core/blank-down",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{ISBN_Type}",
                "description": "Blank down cells in column ``{ISBN_Type}``"
            }}
        """
        For_OpenRefine.write(Template_String)
    For_OpenRefine.write("]")

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

Copy Remove_Blank_Rows_After_Pivot.json and paste into OpenRefine

# Organize HOL

> Start by loading Alpeh Sequential text file into OpenRefine with column widths 10,5,3

Put the unique name of the OpenRefine project for the HOL records in the variable `HOL_Project_Name` and run the cell

In [337]:
HOL_Project_Name = "oso_hol_misc_2021 05 02 txt"

Copy Organize_Aleph_Sequential_for_HOL.json and paste into OpenRefine

Open a text filter for "Fields", click on the number of choices to get the facet choices as TSV, paste into variable `Number_of_TKRs` below, and run cell (if there aren't any TKRs, do this same procedure with `Number_of_TKRs` as an empty string)

In [338]:
Number_of_TKRs = """
"""

Copy Pivot_Fields_and_Values.json and paste into OpenRefine

In [339]:
Number_of_TKRs = Number_of_TKRs.split("\n")[1:-1] # The splice removes elements created by having the opening and closing quotes on their own lines
TKR_Fields = []
for TKR in Number_of_TKRs:
    TKR = TKR.rstrip("1234567890")
    TKR = TKR.rstrip("\t")
    TKR_Fields.append(TKR)

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    For_OpenRefine.write("[")
    for TKR_Field in TKR_Fields:
        Template_String = f"""
            {{
                "op": "core/text-transform",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{TKR_Field}",
                "expression": "grel:row.record.cells[\\"{TKR_Field}\\"].value[0]",
                "onError": "keep-original",
                "repeat": false,
                "repeatCount": 10,
                "description": "Fill up and down values in ``{TKR_Field}``"
            }},
            {{
                "op": "core/blank-down",
                "engineConfig": {{
                "facets": [],
                "mode": "record-based"
                }},
                "columnName": "{TKR_Field}",
                "description": "Blank down cells in column ``{TKR_Field}``"
            }}
        """
        For_OpenRefine.write(Template_String)
    For_OpenRefine.write("]")

Run the cell above, then copy JSON_Output.txt and paste into OpenRefine

**Need to determine what to do when multiple HOL for same BIB happen**

Copy Organize_TKRs.json and paste into OpenRefine


## Add HOL data to BIB project
Switch back to BIB project and set `i` to position of first identifier column

Run all cells, then copy JSON_Output.txt and paste into OpenRefine

In [340]:
i = 4

with open('JSON_Output.txt', 'w') as For_OpenRefine:
    Template_String = f"""
        [
            {{
                "op": "core/column-addition",
                "engineConfig": {{
                    "facets": [],
                    "mode": "row-based"
                }},
                "baseColumnName": "SYS Number",
                "expression": "grel:cell.cross(\\"{HOL_Project_Name}\\",\\"BIB Number\\").cells[\\"HOL Sublibrary\\"].value[0]",
                "onError": "set-to-blank",
                "newColumnName": "Sublibrary",
                "columnInsertIndex": {i},
                "description": "Create column ``Sublibrary`` with the sublibrary value from row with same BIB in HOL project"
            }},
            {{
                "op": "core/column-addition",
                "engineConfig": {{
                    "facets": [],
                    "mode": "row-based"
                }},
                "baseColumnName": "SYS Number",
                "expression": "grel:cell.cross(\\"{HOL_Project_Name}\\",\\"BIB Number\\").cells[\\"SYS Number\\"].value[0]",
                "onError": "set-to-blank",
                "newColumnName": "HOL Number",
                "columnInsertIndex": {i+1},
                "description": "Create column ``HOL Number`` with the HOL number from row with same BIB in HOL project"
            }},
            {{
                "op": "core/column-addition",
                "engineConfig": {{
                    "facets": [],
                    "mode": "row-based"
                }},
                "baseColumnName": "SYS Number",
                "expression": "grel:cell.cross(\\"{HOL_Project_Name}\\",\\"BIB Number\\").cells.Suppressed.value[0]",
                "onError": "set-to-blank",
                "newColumnName": "Suppressed",
                "columnInsertIndex": {i+2},
                "description": "Create column ``Suppressed`` with the value from column of the same name in the row with same BIB in HOL project"
            }},
    """
    For_OpenRefine.write(Template_String)
    for TKR_Field_Number in range(len(TKR_Fields)):
        Template_String = f"""
            {{
                "op": "core/column-addition",
                "engineConfig": {{
                    "facets": [],
                    "mode": "row-based"
                }},
                "baseColumnName": "SYS Number",
                "expression": "grel:cell.cross(\\"{HOL_Project_Name}\\",\\"BIB Number\\").cells[\\"TKR {TKR_Field_Number+1}\\"].value[0]",
                "onError": "set-to-blank",
                "newColumnName": "TKR {TKR_Field_Number+1}",
                "columnInsertIndex": {TKR_Field_Number+i},
                "description": "Create column ``TKR {TKR_Field_Number+1}`` with the value from the TKR column of the same number from row with same BIB in HOL project"
            }}
        """
        For_OpenRefine.write(Template_String)
    For_OpenRefine.write("]")